In [ ]:
import pymysql
import os 
from dotenv import load_dotenv

load_dotenv()

conn = pymysql.connect(

host=os.getenv("MYSQL_HOST"),       
user=os.getenv("MYSQL_USER_NAME"),
password=os.getenv("MYSQL_USER_PASSWORD"),
db=os.getenv("MYSQL_DATABASE"),
port=int(os.getenv("MYSQL_PORT")),

charset='utf8mb4',
cursorclass=pymysql.cursors.DictCursor
)

In [3]:
query = """
SELECT *
FROM menstable_test2;
"""

with conn.cursor() as cursor:
    cursor.execute(query)
    result = cursor.fetchall()

In [5]:
import pandas as pd 

data = pd.DataFrame(result)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31302 entries, 0 to 31301
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   idx            31302 non-null  int64         
 1   style          31302 non-null  object        
 2   season         31302 non-null  object        
 3   fit            31302 non-null  object        
 4   color          31302 non-null  object        
 5   goods_name     31302 non-null  object        
 6   thumbnail_url  31302 non-null  object        
 7   is_soldout     31302 non-null  int64         
 8   goods_url      31302 non-null  object        
 9   brand          31302 non-null  object        
 10  normal_price   31302 non-null  int64         
 11  price          31302 non-null  int64         
 12  main_category  31302 non-null  object        
 13  sub_category   31302 non-null  object        
 14  created_at     31302 non-null  datetime64[ns]
 15  updated_at     3130

In [6]:
# s3 버킷 

import boto3 
from botocore.exceptions import ClientError

service_name = os.getenv("service_name")
endpoint_url = os.getenv("endpoint_url")
region_name = os.getenv("region_name")
access_key = os.getenv("access_key")
secret_key = os.getenv("secret_key")

s3 = boto3.client(service_name, endpoint_url=endpoint_url, aws_access_key_id=access_key,
                  aws_secret_access_key=secret_key,
                  config=boto3.session.Config(signature_version="s3v4"))

In [62]:
import boto3

bucket_name = "web-assets-prod"
prefix = "clothes/"

paginator = s3.get_paginator("list_objects_v2")
page_iterator = paginator.paginate(Bucket=bucket_name, Prefix=prefix)

all_keys = []

for page in page_iterator:
    contents = page.get("Contents", [])
    for obj in contents:
        all_keys.append(obj["Key"])


In [64]:
len(all_keys)

25977

In [3]:
all_keys[:-1]

NameError: name 'all_keys' is not defined

In [ ]:
s3_keys = [obj["Key"] for obj in all_keys.get("Contents", [])]

AttributeError: 'list' object has no attribute 'get'

In [61]:
len(s3_keys[1:])

999

In [ ]:
s3_map = {
    key.split("/")[-1].replace(".jpg", ""): key 
    for key in all_keys[1:]
}

In [75]:
len(s3_map)

25976

In [76]:
def extract_id(url: str) -> str:
    try:
        return url.split("/")[6]
    except IndexError:
        return ""

In [4]:
s3_map

NameError: name 's3_map' is not defined

In [77]:
data["image_id"] = data["thumbnail_url"].apply(lambda x: str(x).split("/")[6].strip())
data["s3_path"] = data["image_id"].map(s3_map)


In [5]:
for key in list(s3_map.keys())[:10]:
    print(repr(key))


NameError: name 's3_map' is not defined

In [78]:
test_id = data["image_id"].iloc[0]
print("테스트 ID:", test_id)
print("s3_map에 존재?", test_id in s3_map)

테스트 ID: 4471126
s3_map에 존재? True


In [104]:
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

load_dotenv()

# SQLAlchemy 엔진 생성
engine = create_engine(
    f"mysql+pymysql://{os.getenv('MYSQL_USER_NAME')}:{os.getenv('MYSQL_USER_PASSWORD')}@{os.getenv('MYSQL_HOST')}:{os.getenv('MYSQL_PORT')}/{os.getenv('MYSQL_DATABASE')}"
)

# DataFrame을 새 테이블에 업로드
data.to_sql(
    name="mens_table_refine",  # 새 테이블명
    con=engine,
    index=False,
    if_exists="replace"      # 없으면 생성, 있으면 덮어쓰기 ("append"로 바꾸면 추가)
)

print("mens_table_refine테이블로 업로드 완료!")


mens_table_refine테이블로 업로드 완료!


In [94]:
def validate_match(row):
    if pd.isna(row["s3_path"]):
        return False
    return row["image_id"] in row["s3_path"]

data["match"] = data.apply(validate_match, axis=1)


In [98]:
data["match"]

0        True
1        True
2        True
3        True
4        True
         ... 
31297    True
31298    True
31299    True
31300    True
31301    True
Name: match, Length: 31302, dtype: bool

In [ ]:
data["s3_path"] = data["s3_path"].apply(lambda x : "https://kr.object.iwinv.kr/" + x)[0]

In [1]:
data

NameError: name 'data' is not defined

In [105]:
import pandas as pd

query = "SELECT * FROM shoes_refine;"
df = pd.read_sql(query, con=engine)

In [ ]:
df["s3_path"] = df["s3_path"].apply(lambda x : "https://kr.object.iwinv.kr/" + x)

In [2]:
df

NameError: name 'df' is not defined

In [112]:
df.to_sql(
    name="shoes_refine",  # 기존 테이블 덮어쓰기 or 새로 저장
    con=engine,
    index=False,
    if_exists="replace"   # "replace": 기존 테이블 삭제 후 저장 / "append": 데이터 추가
)


2978